In [ ]:
#! /usr/bin/python2

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

import sys
import re

if sys.version_info[0] >= 3:
    unicode = str


def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub(r"^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split(r"\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def pairs_starting_from_word(words, first_word='word'):
    pairs = []
    
    for i, word in enumerate(words[:-1]):
        if (word == first_word):
            pair = '{}_{}'.format(word, words[i+1])
            cnt = 1
            pairs.append((pair, cnt))
        else:
            continue
    return pairs


wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)



# lowercase all words
wiki_lower = wiki.map(lambda words: [x.lower() for x in words])

# find pairs starting from defined word
wiki_pairs = wiki_lower.flatMap(lambda x: pairs_starting_from_word(x, 'narodnaya'))

# filtering empty elements
wiki_pairs = wiki_pairs.filter(lambda x: x != [])

# aggregate counters
wiki_red = wiki_pairs.reduceByKey(lambda a, b: a + b, numPartitions=16)

# sort values by key
wiki_red_sorted = wiki_red.sortByKey()


result = wiki_red_sorted.collect()
for pair, cnt in result:
    print ('{}\t{}'.format(pair, cnt))